In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
plt.rc('font', family = 'Malgun Gothic') # 한글 입력
plt.rcParams['axes.unicode_minus'] = False # 글꼴 변경시 마이너스 표기 가능으로 처리

In [164]:
import pandas as pd
df_all = pd.read_csv('/content/drive/MyDrive/1차프로젝트/colab/주말방문_train.csv',encoding='cp949')
df_sebunryu= pd.read_csv('/content/drive/MyDrive/1차프로젝트/colab/주말방문train.csv',encoding='cp949')
display(df_all.head(),df_sebunryu.head())

,고객번호,총구매액
0,1,71940032
1,2,74036940
2,3,2486049
3,4,12956553
4,6,17784434


,고객번호,세분류명,총구매액
0,1,선매품,52584420
1,1,전문품,503500
2,1,편의품,18852112
3,2,선매품,34113370
4,2,전문품,1978060


In [150]:
df_all.columns = ['고객번호','총구매액']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [165]:
# 고객마다 구매이력이 없는 세분류명의 총구매액을 0으로 채워넣어주기

cusno = df_all['고객번호'].unique().tolist()
cat = df_sebunryu['세분류명'].unique().tolist()


cusno_re = []
cat_re = []


for i in np.arange(3):
  for j in cusno:
      cusno_re.append(j)
      
        
for i in np.arange(int(len(cusno))):
    for j in cat:
        cat_re.append(j)
        
cusno_re.sort()

df_sebunryu_re = pd.DataFrame()
df_sebunryu_re['고객번호'] = cusno_re
df_sebunryu_re['세분류명'] = cat_re
df_sebunryu_re = df_sebunryu_re.sort_values(['고객번호'])
df_sebunryu_re['총구매액'] = 0

In [166]:
df_all_scaled_p = pd.merge(df_sebunryu_re, df_sebunryu, how = 'outer', on = ['고객번호','세분류명']).drop(['총구매액_x'], axis = 1).fillna(0)
df_all_scaled_p['총구매액'] = df_all_scaled_p['총구매액_y'].astype('int64')
df_sebunryu_re = df_all_scaled_p.drop('총구매액_y', axis = 1)

In [167]:
a = df_all.고객번호.tolist()
li = []
for i in a:
    b = float(df_sebunryu_re.loc[(df_sebunryu_re['고객번호']== i)&(df_sebunryu_re['세분류명']=='선매품')]['총구매액'].values)/float(df_all.loc[df_all['고객번호']== i]['총구매액'].values)
    li.append(b)
df_all['선매품구매비중'] = li

In [168]:
li2 = []
for i in a:
    c = float(df_sebunryu_re.loc[(df_sebunryu_re['고객번호']== i)&(df_sebunryu_re['세분류명']=='편의품')]['총구매액'].values)/float(df_all.loc[df_all['고객번호']== i]['총구매액'].values)
    li2.append(c)
df_all['편의품구매비중'] = li2

In [169]:
li3 = []
for i in a:
    d = float(df_sebunryu_re.loc[(df_sebunryu_re['고객번호']== i)&(df_sebunryu_re['세분류명']=='전문품')]['총구매액'].values)/float(df_all.loc[df_all['고객번호']== i]['총구매액'].values)
    li3.append(d)
df_all['전문품구매비중'] = li3

In [170]:
df_fin = pd.read_csv('/content/drive/MyDrive/1차프로젝트/colab/train고객속성변수.csv',index_col=0)

In [171]:
df_fin.head()

,고객번호,성별,CAT연령대,지역,경쟁사이용횟수,채널이용횟수,멤버십가입개수,RECENCY6,고객구분1_7,편의품구매비중,선매품구매비중,전문품구매비중,주말방문비중,주말구매비중
0,1,M,노년층,서울,15,0,0,3,감소,0.0,0.0,0.0,0.253863,0.292974
1,2,M,노년층,경기,8,0,0,6,증가,0.0,0.0,0.0,0.331803,0.357044
2,3,M,노년층,서울,0,0,0,1,증가,0.0,0.0,0.0,0.174140,0.171020
3,4,F,노년층,서울,3,0,0,0,증가,0.0,0.0,0.0,0.234043,0.213483
4,6,F,노년층,강원,0,0,0,0,증가,0.0,0.0,0.0,0.280977,0.192334


In [172]:
df_fin['편의품구매비중'] = df_all['편의품구매비중']
df_fin['선매품구매비중'] = df_all['선매품구매비중']
df_fin['전문품구매비중'] = df_all['전문품구매비중']

In [173]:
df_fin.head()

,고객번호,성별,CAT연령대,지역,경쟁사이용횟수,채널이용횟수,멤버십가입개수,RECENCY6,고객구분1_7,편의품구매비중,선매품구매비중,전문품구매비중,주말방문비중,주말구매비중
0,1,M,노년층,서울,15,0,0,3,감소,0.262053,0.730948,0.006999,0.253863,0.292974
1,2,M,노년층,경기,8,0,0,6,증가,0.512521,0.460761,0.026717,0.331803,0.357044
2,3,M,노년층,서울,0,0,0,1,증가,0.957000,0.043000,0.000000,0.174140,0.171020
3,4,F,노년층,서울,3,0,0,0,증가,0.434418,0.565582,0.000000,0.234043,0.213483
4,6,F,노년층,강원,0,0,0,0,증가,0.611053,0.380231,0.008715,0.280977,0.192334


In [174]:
df_fin.to_csv('/content/drive/MyDrive/1차프로젝트/colab/train고객속성변수.csv')

In [175]:
df = pd.read_csv('/content/drive/MyDrive/1차프로젝트/colab/train고객속성변수.csv',index_col=0)
df

,고객번호,성별,CAT연령대,지역,경쟁사이용횟수,채널이용횟수,멤버십가입개수,RECENCY6,고객구분1_7,편의품구매비중,선매품구매비중,전문품구매비중,주말방문비중,주말구매비중
0,1,M,노년층,서울,15,0,0,3,감소,0.262053,0.730948,0.006999,0.253863,0.292974
1,2,M,노년층,경기,8,0,0,6,증가,0.512521,0.460761,0.026717,0.331803,0.357044
2,3,M,노년층,서울,0,0,0,1,증가,0.957000,0.043000,0.000000,0.174140,0.171020
3,4,F,노년층,서울,3,0,0,0,증가,0.434418,0.565582,0.000000,0.234043,0.213483
4,6,F,노년층,강원,0,0,0,0,증가,0.611053,0.380231,0.008715,0.280977,0.192334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18861,19333,F,청년층,서울,0,5,1,0,감소,0.983143,0.016857,0.000000,0.353111,0.361289
18862,19334,M,중년층,충남,30,0,1,7,감소,0.906420,0.093580,0.000000,0.299363,0.292447
18863,19335,F,중년층,전북,0,0,1,1,증가,0.565545,0.434455,0.000000,0.237099,0.294739
18864,19336,F,중년층,서울,0,0,0,5,증가,0.333447,0.645958,0.020595,0.420118,0.474068
